In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [1]:
!python -V

Python 3.10.11


In [8]:
import pickle
import pandas as pd
import numpy as np

In [11]:
year = 2023
month = 3 # March

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
np.std(y_pred)

df.index

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       3403756, 3403757, 3403758, 3403759, 3403760, 3403761, 3403762, 3403763,
       3403764, 3403765],
      dtype='int64', length=3316216)

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = df[['ride_id']].copy()
df_result['prediction'] = y_pred
output_file = 'predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)